## JG Thoughts 
- Unemployment rate vs employment #s? Look at change in UE rate or total as of certain point in time
- Inc_Per_CAp_2022 -> typo?
- Population Density per mi² -> what yr? Also is string
- NA values in NJ
- Convert weeks into %?

- Maybe PCA for the white, black, Hispanic stats? But missing a decent chunk it looks like. Maybe only black or white. But a lot of 0s so maybe just exclde entirely  
- Maybe do something to 'Population' like catgy 1-5 for large small or medium. But can leave for now I guess 

- Do SVM or NN models? SVM mostly for classification. NN maybe not enough data

## Data Cleaning & Preprocessing

In [375]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.ensemble import VotingRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor


In [328]:
df = pd.read_csv('Data/County_Data_needsVAXmerge.csv')
df.head()

,Unnamed: 0,FIPS,State,County,Years of Potential Life Lost Rate (premature death),YPLL Rate (Black),YPLL Rate (Hispanic),YPLL Rate (White),% Fair/Poor Health,percent_smokers,...,percent Not Proficient in English,percent Female,number Rural,cases_2020,cases_2021,cases_2022,deaths_2020,deaths_2021,deaths_2022,Masks
0,0,1001,Alabama,Autauga,8824.0,10471.0,0.0,8707.0,18,19,...,1,51.3,22921.0,4190.0,11018.0,18961.0,5631.0,47405.0,124934.0,267.0
1,1,1003,Alabama,Baldwin,7225.0,10042.0,3087.0,7278.0,18,17,...,0,51.5,77060.0,13601.0,39911.0,67496.0,12412.0,148723.0,397246.0,267.0
2,2,1005,Alabama,Barbour,9586.0,11333.0,0.0,7310.0,26,22,...,1,47.2,18613.0,1514.0,3860.0,7027.0,2035.0,24364.0,60044.0,267.0
3,4,1007,Alabama,Bibb,11784.0,14813.0,0.0,11328.0,20,20,...,0,46.5,15663.0,1834.0,4533.0,7692.0,2678.0,28085.0,65964.0,267.0
4,5,1009,Alabama,Blount,10908.0,0.0,5620.0,11336.0,21,20,...,2,50.7,51562.0,4641.0,11256.0,17731.0,3855.0,56300.0,144559.0,267.0


In [329]:
# Drop rows that we will not be using
df.drop(columns = ['Unnamed: 0', 'County', 'YPLL Rate (Black)', 'YPLL Rate (Hispanic)', 'YPLL Rate (White)', 'Number Uninsured', 'Number Primary Care Physicians', 
                        'Preventable Hosp. Rate (Black)', 'Preventable Hosp. Rate (Hispanic)', 'Preventable Hosp. Rate (White)',  'Percent Vaccinated Flu (Black)', 
                        'Percent  Vaccinated (Hispanic) Flu', 'Percent Vaccinated (White) Flu', 'Number Some College', 'Number Unemployed', 'Labor Force', 'PCP Ratio',
                        '80th Percentile Income', '20th Percentile Income', '95% CI - Low', '95% CI - High', 'Life Expectancy (Black)', 'Life Expectancy (Hispanic)', 
                        'Life Expectancy (White)', 'Number HIV Cases', 'Household income (Black)', 'Household income (Hispanic)', 'Household income (White)'], inplace = True)

In [330]:
# df.columns
# df.dtypes
# df.shape

In [331]:
# Make FIPS index 
df.set_index('FIPS', inplace=True)

In [332]:
# Create new columns for per populaltion stats - YPL, Number pre-mature Deaths, Number rural 
df['YPL'] = df['Years of Potential Life Lost Rate (premature death)']/df['Population']
df['pre mature deaths'] = df['Number pre-mature Deaths']/df['Population']
df['rural'] = df['number Rural']/df['Population']

df.drop(columns = ['Years of Potential Life Lost Rate (premature death)', 'Number pre-mature Deaths', 'number Rural'], inplace = True)

In [333]:
# Dummify State and Presence of water violation
df = pd.get_dummies(columns = ['State'], data = df, drop_first=True)
df['water'] = df['Presence of water violation'].map({'No': 0, 'Yes': 1})
df.drop(columns = ['Presence of water violation'], inplace = True)

In [334]:
# Calculate total cases and deaths, and convert to % of population 
df['cases'] = df['cases_2020'] + df['cases_2021'] + df['cases_2022']
df['deaths'] = df['deaths_2020'] + df['deaths_2021'] + df['deaths_2022']

df['case_rate'] = df['cases']/df['Population']
df['death_rate'] = df['deaths']/df['Population']

# Deaths seem to be off? More deaths than population 
df.drop(columns = ['cases_2020', 'cases_2021', 'cases_2022', 'deaths_2020', 'deaths_2021', 'deaths_2022', 'cases', 'deaths'], inplace = True)

In [335]:
# Drop Na values (1850 rows -> 1807)
df.dropna(inplace=True)
df.shape

(1807, 97)

## Linear Regression (LR)

In [336]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [337]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [338]:
print(f'Train Score: {lr.score(X_train, y_train)}')
print(f'Test Score: {lr.score(X_test, y_test)}')

Train Score: 0.4386288043517149
Test Score: 0.5169894950868115


In [339]:
pd.DataFrame(set(zip(X.columns, lr.coef_)), columns = ['Variable Name', 'Coefficient']).sort_values('Coefficient')

,Variable Name,Coefficient
23,State_Alaska,-0.518679
1,State_Louisiana,-0.369001
42,State_California,-0.330751
34,State_Utah,-0.282775
44,State_Oregon,-0.276141
...,...,...
2,Percent Uninsured children,0.017699
83,percent Native Hawaiian/Other Pacific Islander,0.044031
68,State_Tennessee,0.081605
64,State_Mississippi,0.103736


## K Nearest Neighbors (KNN)

In [340]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [341]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [342]:
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train_sc, y_train)

KNeighborsRegressor(n_neighbors=10)

In [343]:
print(f'Train Score: {knn.score(X_train_sc, y_train)}')
print(f'Test Score: {knn.score(X_test_sc, y_test)}')

Train Score: 0.42261667295082517
Test Score: 0.5508477207731012


## Decision Trees (DT)

In [344]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [345]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [346]:
print(f'Train Score: {tree.score(X_train, y_train)}')
print(f'Test Score: {tree.score(X_test, y_test)}')

Train Score: 1.0
Test Score: -0.580270130389847


In [347]:
# Top 10 features
pd.DataFrame({'feature': X.columns, 'importance': tree.feature_importances_}).sort_values('importance', ascending = False).head(10)

,feature,importance
6,Percent Uninsured,0.243239
29,Population,0.127059
44,State_Alaska,0.099993
61,State_Louisiana,0.086535
30,percent < 18,0.075028
53,State_Georgia,0.031041
31,percent 65 and over,0.028498
26,Median Household Income,0.027373
40,Masks,0.023360
39,percent Female,0.019220


### Bagged Decision Tree

In [351]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [352]:
pgrid = {
    'estimator__max_depth': [1, 10, 20, 30],
    'estimator__min_samples_leaf': [1, 10, 20, 30]
}

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
tree_bagged = GridSearchCV(bag, pgrid, cv = 5, n_jobs = 6)
bag = BaggingRegressor(tree, n_estimators = 100, random_state = 42)
# tree_bagged.get_params()

In [353]:
tree_bagged.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=BaggingRegressor(estimator=DecisionTreeRegressor(),
                                        n_estimators=100, random_state=42),
             n_jobs=6,
             param_grid={'estimator__max_depth': [1, 10, 20, 30],
                         'estimator__min_samples_leaf': [1, 10, 20, 30]})

In [354]:
print(f'Train Score: {tree_bagged.score(X_train, y_train)}')
print(f'Test Score: {tree_bagged.score(X_test, y_test)}')

Train Score: 0.4980347697824272
Test Score: 0.5078247751975666


In [277]:
tree_bagged.best_params_

{'estimator__max_depth': 20, 'estimator__min_samples_leaf': 10}

## Random Forest (RF)

In [367]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [368]:
rf = RandomForestRegressor(
    n_estimators=500,
    random_state = 42,
)

In [369]:
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=500, random_state=42)

In [370]:
print(f'Train Score: {rf_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_rs.score(X_test, y_test)}')

Train Score: 0.5993254668253245
Test Score: 0.5135590706563831


In [371]:
pd.DataFrame({'Features': X.columns, 'Importance': rf.feature_importances_}).sort_values('Importance', ascending = False)

,Features,Importance
61,State_Louisiana,0.111679
6,Percent Uninsured,0.095644
29,Population,0.086377
14,Average Daily PM2.5,0.061007
44,State_Alaska,0.052688
...,...,...
72,State_New Hampshire,0.000009
49,State_Connecticut,0.000005
50,State_Delaware,0.000003
54,State_Hawaii,0.000000


### RF w/ RandomSearchCV

In [ ]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [359]:
params = {
    'max_features': np.arange(10, X.shape[1] + 1),
    'min_samples_leaf': [1, 5, 10, 15]
}

rf = RandomForestRegressor(
    n_estimators=500,
    random_state = 42,
)


rf_rs = RandomizedSearchCV(rf, params, n_iter=20, cv = 5, n_jobs = 6)
# rf_rs.get_params()

In [360]:
rf_rs.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_estimators=500,
                                                   random_state=42),
                   n_iter=20, n_jobs=6,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95]),
                                        'min_samples_leaf': [1, 5, 10, 15]})

In [363]:
print(f'Train Score: {rf_rs.score(X_train, y_train)}')
print(f'Test Score: {rf_rs.score(X_test, y_test)}')

Train Score: 0.5993254668253245
Test Score: 0.5135590706563831


In [364]:
rf_rs.best_params_

{'min_samples_leaf': 5, 'max_features': 61, 'max_depth': None}

### RF w/ Adaboost

In [377]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [378]:
rf_ada = AdaBoostRegressor(
    RandomForestRegressor(),
    n_estimators = 500
)    

In [379]:
rf_ada.fit(X_train, y_train)

AdaBoostRegressor(estimator=RandomForestRegressor(), n_estimators=500)

In [380]:
print(f'Train Score: {rf_ada.score(X_train, y_train)}')
print(f'Test Score: {rf_ada.score(X_test, y_test)}')

Train Score: 0.980491688155413
Test Score: 0.3097373298902043


### RF w/ Gradboost

In [381]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [382]:
rf_gb = GradientBoostingRegressor()

In [383]:
rf_gb.fit(X_train, y_train)

GradientBoostingRegressor()

In [384]:
print(f'Train Score: {rf_gb.score(X_train, y_train)}')
print(f'Test Score: {rf_gb.score(X_test, y_test)}')

Train Score: 0.8157639309300915
Test Score: 0.15741888943544657


## Extra Trees (ET)

In [372]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [373]:
et = ExtraTreesRegressor(n_estimators = 500)
et.fit(X_train, y_train)

ExtraTreesRegressor(n_estimators=500)

In [374]:
print(f'Train Score: {et.score(X_train, y_train)}')
print(f'Test Score: {et.score(X_test, y_test)}')

Train Score: 1.0
Test Score: 0.2133695588783927


## Ensembled Model 

In [385]:
# y variable will be case rate or death rate
y = df['case_rate']
# y = df['death_rate']

# X variables
X = df.drop(columns = ['case_rate', 'death_rate'])

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [386]:
vr = VotingRegressor([
    ('lr', lr),
    ('knn', knn),
    ('tree', tree_bagged),
    ('rf', rf),
    ('rf_rs', rf_rs),
    ('rf_ada', rf_ada),
    ('rf_gb', rf_gb),
    ('et', et)
])

In [388]:
vr.fit(X_train, y_train)

In [ ]:
print(f'Train Score: {vr.score(X_train, y_train)}')
print(f'Train Score: {vr.score(X_test, y_test)}')